# 4. Inference

In [1]:
import sys, os
import torch
import pandas as pd
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")
import setting

from Clust.clust.ML.common.common import p1_integratedDataSaving as p1
from Clust.clust.ML.tool import data as ml_data
from Clust.clust.ML.tool import scaler as ml_scaler
from Clust.clust.ML.tool import clean as ml_clean
import pathSetting

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

/Users/kim-youngkee/Documents/CLUSTER/.clust/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu is available.


In [2]:
# Set regression mode i.e., 'regression','forecast' 
mode_selection = "forecast"

### 4-1. Get model meta by mongodb

In [3]:
from Clust.clust.ingestion.mongo.mongo_client import MongoClient
from Clust.setting import influx_setting_KETI as ins

mongo_client = MongoClient(ins.CLUSTMetaInfo2)

db_name = 'model'
collection_name = 'meta'

all_model_meta = mongo_client.get_all_document(db_name, collection_name)

In [4]:
search = {'modelName': 'TestLSTM_rg'}
model_meta= mongo_client.get_document_by_json(db_name, collection_name, search)
model_meta = model_meta[-1]

In [5]:
model_meta

{'trainDataInfo': {'startTime': '2016-01-11',
  'endTime': '2016-04-15',
  'dataInfo': [['life_indoor_environment', 'humidityTrain_10min'],
   ['life_indoor_environment', 'temperatureTrain_10min'],
   ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']],
  'processParam': {'refine_param': {'removeDuplication': {'flag': False},
    'staticFrequency': {'flag': False, 'frequency': None}},
   'outlier_param': {'certainErrorToNaN': {'flag': False},
    'unCertainErrorToNaN': {'flag': False, 'param': {}}},
   'imputation_param': {'flag': False,
    'imputation_method': [],
    'totalNonNanRatio': 80}},
  'integration_freq_sec': 600,
  'cleanParam': 'NoClean',
  'DataSaveMode': 'influx'},
 'modelName': 'TestLSTM_rg',
 'featureList': ['RH_1',
  'RH_2',
  'RH_3',
  'RH_4',
  'RH_5',
  'RH_6',
  'RH_7',
  'RH_8',
  'RH_9',
  'T1',
  'T2',
  'T3',
  'T4',
  'T5',
  'T6',
  'T7',
  'T8',
  'T9',
  'Press_mm_hg',
  'RH_out',
  'T_out',
  'Tdewpoint',
  'Visibility',
  'Windspeed']

### 4-2. Inference data selection

In [6]:
# 0. pick only one data
DataMeta = p1.read_json_data(pathSetting.DataMetaPath)
dataList =  list(DataMeta.keys())

In [7]:
dataList

['IntegraionTrainX',
 'IntegraionTrainy',
 'IntegraionTestX',
 'IntegraionTesty',
 'trainClean_Hs1SwineFarmWithWeatherTime',
 'trainNoClean_Hs1SwineFarmWithWeatherTime',
 'testClean_Hs1SwineFarmWithWeatherTime',
 'testNoClean_Hs1SwineFarmWithWeatherTime']

In [8]:
if mode_selection == 'regression':
    # dataX
    dataName_X = dataList[2]
    dataSaveMode_X = DataMeta[dataName_X]['integrationInfo']['DataSaveMode']

    dataFolderName = "data_integrated_result"
    current = os.getcwd()
    dataFolderPath = os.path.join(current, dataFolderName)

    window_num =144
    # dataX = ml_data.get_saved_integrated_data(dataSaveMode_X, dataName_X, dataFolderPath)[:window_num]
    dataX = ml_data.get_saved_integrated_data('CSV', dataName_X, dataFolderPath)[:window_num]

elif mode_selection == 'forecast':
    LearningModeList = ['train', 'test']
    LearningMode = LearningModeList[1]

    cleanParamList = ['Clean', 'NoClean']
    cleanMode = cleanParamList[1]

    datasetNameList = ['Hs1SwineFarmWithWeatherTime', 'gunwiStrawberryWithWeatherTime', 'strawberryOpenTime']
    datasetName = datasetNameList[0]

    dataName_X = LearningMode + cleanMode + '_' + datasetName
    dataSaveMode_X = DataMeta[dataName_X]['integrationInfo']['DataSaveMode']

    dataX = ml_data.get_saved_integrated_data(dataSaveMode_X, dataName_X, pathSetting.dataFolderPath)
    integration_freq_sec = DataMeta[dataName_X]['integrationInfo']['integration_freq_sec']

In [9]:
print(dataX)

                                RH_1       RH_2       RH_3       RH_4  \
datetime                                                                
2021-01-01 00:00:00+00:00  40.260000  43.200000  38.530000  37.363333   
2021-01-01 00:10:00+00:00  40.260000  43.163333  38.500000  37.230000   
2021-01-01 00:20:00+00:00  40.290000  42.963333  38.633333  37.200000   
2021-01-01 00:30:00+00:00  40.290000  42.490000  38.790000  37.200000   
2021-01-01 00:40:00+00:00  40.626667  42.156667  38.596667  37.090000   
...                              ...        ...        ...        ...   
2021-01-01 23:10:00+00:00  41.500000  42.090000  38.290000  40.730000   
2021-01-01 23:20:00+00:00  41.833333  42.290000  38.290000  40.790000   
2021-01-01 23:30:00+00:00  42.090000  42.363333  38.290000  41.166667   
2021-01-01 23:40:00+00:00  42.030000  42.433333  38.363333  42.093333   
2021-01-01 23:50:00+00:00  41.900000  42.500000  38.290000  42.400000   

                                RH_5       RH_6   

### 4-3. Inference data preparation

In [10]:
feature_list = model_meta['featureList']
target = model_meta['target']
scaler_param = model_meta['scalerParam']
model_file_path = model_meta['files']['modelFile']['filePath']
model_method = model_meta['model_method']
# train_parameter = model_meta["trainParameter"]

if mode_selection == 'regression':
    # Scaling Inference Input
    X_scaler_file_path = model_meta['files']['XScalerFile']['filePath']
    y_scaler_file_path = model_meta['files']['yScalerFile']['filePath']

    infer_X, scaler_X = ml_scaler.get_scaled_test_data(dataX[feature_list], X_scaler_file_path, scaler_param)
    scaler_y = ml_scaler.get_scaler_file(y_scaler_file_path)

elif mode_selection == 'forecast':
    clean_param = model_meta['cleanTrainDataParam']
    nan_processing_param = model_meta['NaNProcessingParam']

    # only for forecast data?
    # past_step = transform_param['past_step']
    # feature_data = dataX[feature_list]
    # step_data = feature_data[-past_step:][feature_list].values
    # df_data = pd.DataFrame(step_data, columns=feature_list)

    # Scaling Inference Input
    X_scaler_file_path = model_meta['files']['XScalerFile']['filePath']
    infer_X, scaler_X = ml_scaler.get_scaled_test_data(dataX[feature_list], X_scaler_file_path, scaler_param)
    clean_infer_X = ml_clean.get_cleand_data(infer_X, clean_param, integration_freq_sec, nan_processing_param)

In [11]:
# print(type(clean_infer_X), clean_infer_X.shape)
# print(clean_infer_X)
print(type(infer_X), infer_X.shape)
print(infer_X)

<class 'pandas.core.frame.DataFrame'> (144, 24)
                               RH_1      RH_2      RH_3      RH_4      RH_5  \
datetime                                                                      
2021-01-01 00:00:00+00:00  0.364279  0.664944  0.456302  0.386490  0.275336   
2021-01-01 00:10:00+00:00  0.364279  0.663872  0.454900  0.380531  0.275336   
2021-01-01 00:20:00+00:00  0.365104  0.658023  0.461131  0.379190  0.274333   
2021-01-01 00:30:00+00:00  0.365104  0.644180  0.468453  0.379190  0.274333   
2021-01-01 00:40:00+00:00  0.374369  0.634432  0.459417  0.374274  0.274333   
...                             ...       ...       ...       ...       ...   
2021-01-01 23:10:00+00:00  0.398404  0.632482  0.445085  0.536963  0.335480   
2021-01-01 23:20:00+00:00  0.407577  0.638331  0.445085  0.539644  0.331521   
2021-01-01 23:30:00+00:00  0.414641  0.640476  0.445085  0.556479  0.328413   
2021-01-01 23:40:00+00:00  0.412990  0.642523  0.448512  0.597896  0.326960   
2021

In [12]:
def transDFtoNP_inf_yk(dfX, windowNum = 0, dim = None):
    """
    Make NumpyArray by input DataFrame.
    if windowNum = 0 ----> slice X by day
    if windowNum = N ----> slice X by windowNum
    
    Example:
        >>> Retunrn 
        ... X.shape (sampleNum, featureNum, sequenceNum )

    Args:
        dfX (DataFrame): dfX
        windowNum (Interger): windowNum

    Returns:
        numpy array:  X
    
    """
    import datetime as dt
    import numpy as np

    if dim == 2:
        X = dfX.to_numpy()
    else:
        X =[]

        if windowNum ==0:
            dateList = dfX.index.map(lambda t: t.date()).unique()
            for startDate in dateList:
                endDate  = dt.datetime.combine(startDate, dt.time(23, 59, 59, 59))
                dfX_partial = dfX[startDate:endDate]
                X_partial = dfX_partial.values
                X.append (X_partial)
        else:
            import math
            roundNum = math.ceil(len(dfX)/windowNum)
            for i in range(roundNum): #This ensures all rows are captured
                dfX_partial = dfX[i*windowNum:(i+1)*windowNum]
                X_partial = dfX_partial.values
                X.append (X_partial)

        X = np.array(X)
    
    return X

In [13]:
transformParameter = model_meta['transformParameter']
# # transform
if mode_selection == 'regression':

    # from Clust.clust.transformation.type.DFToNPArray import transDFtoNP, trans_df_to_np, trans_df_to_np_inf
    inferX = transDFtoNP_inf_yk(infer_X)

# forecast
elif mode_selection == 'forecast':
    
    from Clust.clust.transformation.purpose.machineLearning import LSTMData
    LSTMD = LSTMData()
    inferX, _ = LSTMD.transform_Xy_arr(clean_infer_X, transformParameter, transformParameter['clean_param'])

/var/folders/gs/pbqr84y96p71g7j8lmqfh1r00000gn/T/ipykernel_46313/2120287005.py:31: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  dfX_partial = dfX[startDate:endDate]


In [14]:
print(type(inferX), inferX.shape)
print(inferX)

<class 'numpy.ndarray'> (1, 144, 24)
[[[0.36427851 0.66494443 0.4563016  ... 0.48868778 0.609375   0.30769231]
  [0.36427851 0.6638721  0.45489952 ... 0.47963801 0.609375   0.29487179]
  [0.36510412 0.65802301 0.46113102 ... 0.47058824 0.609375   0.28205128]
  ...
  [0.41464086 0.64047573 0.4450849  ... 0.67420814 0.609375   0.53846154]
  [0.41298963 0.64252291 0.44851223 ... 0.67571644 0.609375   0.51282051]
  [0.40941198 0.64447261 0.4450849  ... 0.67722474 0.609375   0.48717949]]]


In [15]:
modelParameter = model_meta["modelParameter"]

inferParameter = {
    'device': 'cpu'
}

In [16]:
# 4. Inference
from Clust.clust.ML.regression_YK.inference import RegressionInference as RI

ri = RI()
ri.set_param(inferParameter)
ri.set_model(model_method, model_file_path, modelParameter)
ri.set_data(inferX)
preds = ri.inference()
print(preds)

inference data shape: torch.Size([1, 144, 24])

Start inference

** Dimension of result for inference dataset = (1,)
[0.22638921]


In [17]:
print(len(preds))

1


In [18]:
# for regression
if mode_selection == 'regression':
    if scaler_param =='scale':
        base_df_for_inverse = pd.DataFrame(columns=target, index=range(len(preds)))
        base_df_for_inverse[target] = preds
        prediction_result = pd.DataFrame(scaler_y.inverse_transform(base_df_for_inverse), columns=target, index=base_df_for_inverse.index)
    else:
        prediction_result = pd.DataFrame(data={"value": preds}, index=range(len(preds)))

# for forecast
elif mode_selection == 'forecast':
    if scaler_param =='scale':
        base_df_for_inverse = pd.DataFrame(columns=feature_list, index=range(len(preds)))
        base_df_for_inverse[target] = preds
        inverse_result = pd.DataFrame(scaler_X.inverse_transform(base_df_for_inverse), columns=feature_list, index=base_df_for_inverse.index)
        target_data = inverse_result[target]
        prediction_result = pd.DataFrame(data={target: target_data}, index=range(len(preds)))
    else:
        prediction_result = pd.DataFrame(data={target: preds}, index=range(len(preds)))

In [19]:
prediction_result

,value
0,10.08059
